# Web Scraper Tool for 5 US Media Outlets

Import packages

In [55]:
# import packages
import requests
from bs4 import BeautifulSoup

import pandas as pd

### 1. Breitbart - Very Conservative

Get links from the Breitbart homepage

### 2. Fox - Conservative

Get links from the Fox Politics homepage

In [56]:
# https://www.foxnews.com/category/politics/2020-presidential-election <- 127 links // 15  without junk
# https://www.foxnews.com/politics <- 150 links (but junk?) // 31 without junk
# i also tried running this for each candidate's section but there is overlap in articles

# load the HTML content using requests and save into a variable
r3 = requests.get('https://www.foxnews.com/politics')
homepage3 = r3.content

# create a soup to allow BeautifulSoup to work
soup3 = BeautifulSoup(homepage3, 'html.parser')

# locate and retrieve all links - WAS NOT ABLE TO ISOLATE JUST ARTICLE LINKS BECUASE NO UNIQUE TAG ELEMENT!!
# you can see how complex the homepage is: print(soup3.prettify())
homepage_tags3 = soup3.find_all('a')
homepage_links3 = []

for link in homepage_tags3:
    homepage_links3.append(link.get('href'))

# remove duplicates by turning list into a set
homepage_links3 = set(homepage_links3)

In [57]:
# remove junk links from list <- MAYBE THIS COULD BE DONE IN A BETTER WAY??
# creating a list of exact junk links that will never be needed when pulling from the homepage
junk_links3 = ['/', '/us', '/world', '/opinion', '/politics', '/entertainment', '//www.foxbusiness.com', 
                  '/lifestyle', '/shows', '//www.foxnews.com/shows/fox-nation', '//radio.foxnews.com/podcast',
                  '#', '//foxnews.com/weather/your-weather/index.html', '#', '#', 
                  '//video.foxnews.com/v/5614615980001/?#sp=watch-live', '#', '/us', '/world', '/opinion', '/politics', 
                  '/official-polls', '/category/politics/elections', 
                  '/entertainment', '//video.foxnews.com/playlist/entertainment-latest-entertainment/', 
                  '//www.foxbusiness.com/', '//www.foxbusiness.com/markets', '//www.foxbusiness.com/politics', 
                  '//www.foxbusiness.com/category/technology', '//www.foxbusiness.com/features', 
                  '//www.foxbusiness.com/category/business-leaders', '/lifestyle', '/food-drink', '/auto', 
                  '/travel', '/family', '/science', '/tech', '/health', '/shows', '/shows', 
                  '/person/personalities', '//video.foxnews.com/v/5614615980001/?#sp=watch-live', 
                  '//video.foxnews.com/playlist/episodic-most-recent-episodes/', 
                  '//video.foxnews.com/#sp=show-clips', '//video.foxnews.com/#sp=news-clips', 
                  '//www.foxnews.com/contact', '//foxcareers.com/Search/SearchResults?brand=Fox%20News%20Careers', 
                  '/foxaroundtheworld/', 'mailto:adsales@foxnews.com?subject=Advertising%20Inquiry', 
                  '//press.foxnews.com/media-contacts/', '//press.foxnews.com/', '/compliance', 
                  'https://supplierdiversity.foxnews.com/', '//www.foxnews.com/shows/fox-nation', '//shop.foxnews.com', 
                  '/go', '//radio.foxnews.com/', '/alerts/subscribe', '/newsletter-signup/alerts', '//radio.foxnews.com/podcast', 
                  '/apps-products', '//www.foxnews.com', '/terms-of-use', '/privacy-policy', '/donotsell', '/closed-captioning', 
                  '//help.foxnews.com', '/contact', '//www.facebook.com/FoxNews', '//twitter.com/foxnews', '//www.google.com/+FoxNews', 
                  '//www.instagram.com/foxnews', '/about/rss/', '/alerts/subscribe', 'https://flipboard.com/@FoxNews', 
                  '//www.foxnews.com/rss/index.html', '//www.foxnews.com/alerts/subscribe.html', 
                   '/accessibility-statement', 'https://video.foxnews.com/v/' ]

# removing all links that lead to section pages AND other specified junk URLs
homepage_links3 = [x for x in homepage_links3 if "/category" not in x] 
homepage_links3 = [x for x in homepage_links3 if "/v/" not in x] 
homepage_links3 = [x for x in homepage_links3 if x not in junk_links3]

In [58]:
# add https://www.foxnews.com to links that don't have this
# create new list for prepared links
hp_links3 = []

for item in homepage_links3:
    new_item = 'https://www.foxnews.com' + item
    hp_links3.append(new_item)

In [60]:
# TO DO: check for duplicates to set of existing URLs to make sure we only pull data for new ones
    # import full dataset in
    # remove urls in homepage_links3 that are already included in the full dataset

Get article text and other relevent data from new articles

In [ ]:
# create a pandas dataframe where all of the retrieved data will be stored
article_data3 = pd.DataFrame()

article_data3[headline] = []
article_data3[date_published] = []
article_data3[date_modified] = []
article_data3[description] = []
article_data3[author] = []
article_data3[url] = []

#export empty df to file
article_data3.to_csv('Fox_Data.csv')

In [211]:
# testing with a real URL
# load the HTML content using requests and save into a variable
r3_article = requests.get('https://www.foxnews.com/politics/mike-bloomberg-suspends-presidential-campaign-after-super-tuesday-show')
article3 = r3_article.content

# create a soups to allow BeautifulSoup to work
article_soup3 = BeautifulSoup(article3, 'html.parser')

# retrieve article specific metadata
metadata3 = article_soup3.find_all("script")[2].get_text()
metadata3 = metadata3.split(",")

article_data3 = []

headline = []
date_published = []
date_modified = []
description = []
author = []
url = []


for item in metadata3:
    if 'headline' in item:
        headline.append(item)
    elif 'datePublished' in item:
        date_published.append(item)
    elif 'dateModified' in item:
        date_modified.append(item)
    elif 'description' in item:
        description.append(item)
    elif 'name' in item and 'Fox News' not in item:
        author.append(item)
    elif 'mainEntityOfPage' in item:
        url.append(item)
        
article_data3.append(headline)
article_data3.append(date_published)
article_data3.append(date_modified)
article_data3.append(description)
article_data3.append(author)

,0
0,"\n ""headline"": ""Mike Bloomberg suspends..."
1,"\n ""datePublished"": ""2020-03-04T10:15:0..."
2,"\n ""dateModified"": ""2020-03-04T11:35:06..."
3,"\n ""description"": ""Former New York City..."
4,"\n ""name"": ""Paul Steinhauser""\n }"


In [ ]:
# import collection of all responses
all_responses = pd.read_csv("Fox_Data.csv", index_col=[0])

# merge new increment of data to the csv
all_responses = all_responses.append(response_df, ignore_index=True)

#export with new increment to save
all_responses.to_csv('Fox_Data.csv')

In [ ]:
# retrieve article text
article_text3 = article_soup3.find_all("script")[2].get_text()

In [144]:
# another spot the author is located just in case
# yay = article_soup3.find_all("meta")[17]
#print(yay)

<meta content="Dana Blanton" data-hid="dc.creator" data-n-head="true" name="dc.creator" scheme="dcterms.creator"/>
